# TABLE OF CONTENTS:
---
* [Notebook Summary](#Notebook-Summary)
* [Setup](#Setup)
    * [Custom Vision](#Custom-Vision)
* [Upload and Tag Data](#Upload-and-Tag-Data)
* [Model Training](#Model-Training)
* [Publish Endpoint](#Publish-Endpoint)
* [Model Evaluation](#Model-Evaluation)
---

# Notebook Summary

This notebook will create a new project in a Custom Vision Azure Cognitive Services resource, upload the stanford dogs training set to it (including class tags), train a model using advanced training and finally evaluate the trained model on the stanford dogs test set.

# Setup

Append parent directory to sys path to be able to import created modules from src directory.

In [1]:
import sys
sys.path.append(os.path.dirname(os.path.abspath("")))

Automatically reload modules when changes are made.

In [2]:
%load_ext autoreload
%autoreload 2

Import libraries and modules.

In [3]:
# Import libraries
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import numpy as np
import os
import time

# Import created modules
from src.utils import EnvVariables

Load environment variables.

In [4]:
env_variables = EnvVariables()

### Custom Vision

Create API key credentials for trainer and predictor.

In [5]:
training_credentials = ApiKeyCredentials(in_headers={"Training-key": env_variables.custom_vision_training_key})
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": env_variables.custom_vision_prediction_key})

Create Custom Vision training and prediction clients.

In [6]:
trainer = CustomVisionTrainingClient(env_variables.custom_vision_endpoint, training_credentials)
predictor = CustomVisionPredictionClient(env_variables.custom_vision_endpoint, prediction_credentials)

Create Custom Vision project.

In [7]:
print ("Creating Custom Vision project...")
project = trainer.create_project(name="Stanford Dogs", classification_type="Multiclass")

Creating Custom Vision project...


# Upload and Tag Data

Upload the training data to the Custom Vision project and tag each image according to its class.

In [8]:
base_image_location_train = "../data/train"

print("Adding images to Custom Vision project...")

for root, dirs, files in os.walk(base_image_location_train):
    for i, file in enumerate(files):
        # For first file in current directory create the class tag in the project
        if i == 0:
            print("=" * 20)
            print(f"New current tag: {root.split('/')[-1]}")
            current_tag = trainer.create_tag(project.id, root.split("/")[-1])
            # Initialize empty image list
            image_list = []
           
        # Read current file (image) and append it to the image_list
        with open(os.path.join(root, file), "rb") as image_contents:
            image_list.append(ImageFileCreateEntry(name=file, contents=image_contents.read(), tag_ids=[current_tag.id]))
            
        # For every 32 files (images) or at the last file (image) of the current directory,
        # upload the batch of images to the project 
        if (i % 31 == 0 and i != 0) or i == (len(files)-1):
            print(f"Upload new batch")
            upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=image_list))
            if not upload_result.is_batch_successful:
                print("Image batch upload failed.")
            for j, image in enumerate(upload_result.images):
                print(f"Image {j+1} of current batch - Image status: ", image.status)
            image_list = []

Adding images to Custom Vision project...
New current tag: n02085620-Chihuahua
Upload new batch
Image 1 of current batch - Image status:  OK
Image 2 of current batch - Image status:  OK
Image 3 of current batch - Image status:  OK
Image 4 of current batch - Image status:  OK
Image 5 of current batch - Image status:  OK
Image 6 of current batch - Image status:  OK
Image 7 of current batch - Image status:  OK
Image 8 of current batch - Image status:  OK
Image 9 of current batch - Image status:  OK
Image 10 of current batch - Image status:  OK
Image 11 of current batch - Image status:  OK
Image 12 of current batch - Image status:  OK
Image 13 of current batch - Image status:  OK
Image 14 of current batch - Image status:  OK
Image 15 of current batch - Image status:  OK
Image 16 of current batch - Image status:  OK
Image 17 of current batch - Image status:  OK
Image 18 of current batch - Image status:  OK
Image 19 of current batch - Image status:  OK
Image 20 of current batch - Image statu

Upload new batch
Image 1 of current batch - Image status:  OK
Image 2 of current batch - Image status:  OK
Image 3 of current batch - Image status:  OK
Image 4 of current batch - Image status:  OK
Image 5 of current batch - Image status:  OK
Image 6 of current batch - Image status:  OK
Image 7 of current batch - Image status:  OK
Image 8 of current batch - Image status:  OK
Image 9 of current batch - Image status:  OK
Image 10 of current batch - Image status:  OK
Image 11 of current batch - Image status:  OK
Image 12 of current batch - Image status:  OK
Image 13 of current batch - Image status:  OK
Image 14 of current batch - Image status:  OK
Image 15 of current batch - Image status:  OK
Image 16 of current batch - Image status:  OK
Image 17 of current batch - Image status:  OK
Image 18 of current batch - Image status:  OK
Image 19 of current batch - Image status:  OK
Image 20 of current batch - Image status:  OK
Image 21 of current batch - Image status:  OK
Image 22 of current batch 

Upload new batch
Image 1 of current batch - Image status:  OK
Image 2 of current batch - Image status:  OK
Image 3 of current batch - Image status:  OK
Image 4 of current batch - Image status:  OK
Image 5 of current batch - Image status:  OK
Image 6 of current batch - Image status:  OK
Image 7 of current batch - Image status:  OK
Image 8 of current batch - Image status:  OK
Image 9 of current batch - Image status:  OK
Image 10 of current batch - Image status:  OK
Image 11 of current batch - Image status:  OK
Image 12 of current batch - Image status:  OK
Image 13 of current batch - Image status:  OK
Image 14 of current batch - Image status:  OK
Image 15 of current batch - Image status:  OK
Image 16 of current batch - Image status:  OK
Image 17 of current batch - Image status:  OK
New current tag: n02086646-Blenheim_spaniel
Upload new batch
Image 1 of current batch - Image status:  OK
Image 2 of current batch - Image status:  OK
Image 3 of current batch - Image status:  OK
Image 4 of cur

Upload new batch
Image 1 of current batch - Image status:  OK
Image 2 of current batch - Image status:  OK
Image 3 of current batch - Image status:  OK
Image 4 of current batch - Image status:  OK
Image 5 of current batch - Image status:  OK
Image 6 of current batch - Image status:  OK
Image 7 of current batch - Image status:  OK
Image 8 of current batch - Image status:  OK
Image 9 of current batch - Image status:  OK
Image 10 of current batch - Image status:  OK
Image 11 of current batch - Image status:  OK
Image 12 of current batch - Image status:  OK
Image 13 of current batch - Image status:  OK
Image 14 of current batch - Image status:  OK
Image 15 of current batch - Image status:  OK
Image 16 of current batch - Image status:  OK
Image 17 of current batch - Image status:  OK
Image 18 of current batch - Image status:  OK
Image 19 of current batch - Image status:  OK
Image 20 of current batch - Image status:  OK
Image 21 of current batch - Image status:  OK
Image 22 of current batch 

Upload new batch
Image 1 of current batch - Image status:  OK
Image 2 of current batch - Image status:  OK
Image 3 of current batch - Image status:  OK
Image 4 of current batch - Image status:  OK
Image 5 of current batch - Image status:  OK
Image 6 of current batch - Image status:  OK
Image 7 of current batch - Image status:  OK
Image 8 of current batch - Image status:  OK
Image 9 of current batch - Image status:  OK
Image 10 of current batch - Image status:  OK
Image 11 of current batch - Image status:  OK
Image 12 of current batch - Image status:  OK
Image 13 of current batch - Image status:  OK
Image 14 of current batch - Image status:  OK
Image 15 of current batch - Image status:  OK
Image 16 of current batch - Image status:  OK
Image 17 of current batch - Image status:  OK
Image 18 of current batch - Image status:  OK
Image 19 of current batch - Image status:  OK
Image 20 of current batch - Image status:  OK
Image 21 of current batch - Image status:  OK
Image 22 of current batch 

...

Upload new batch
Image 1 of current batch - Image status:  OK
Image 2 of current batch - Image status:  OK
Image 3 of current batch - Image status:  OK
Image 4 of current batch - Image status:  OK
Image 5 of current batch - Image status:  OK
Image 6 of current batch - Image status:  OK
Image 7 of current batch - Image status:  OK
Image 8 of current batch - Image status:  OK
Image 9 of current batch - Image status:  OK
Image 10 of current batch - Image status:  OK
Image 11 of current batch - Image status:  OK
Image 12 of current batch - Image status:  OK
Image 13 of current batch - Image status:  OK
Image 14 of current batch - Image status:  OK
Image 15 of current batch - Image status:  OK
Image 16 of current batch - Image status:  OK
Image 17 of current batch - Image status:  OK
New current tag: n02112706-Brabancon_griffon
Upload new batch
Image 1 of current batch - Image status:  OK
Image 2 of current batch - Image status:  OK
Image 3 of current batch - Image status:  OK
Image 4 of cu

Upload new batch
Image 1 of current batch - Image status:  OK
Image 2 of current batch - Image status:  OK
Image 3 of current batch - Image status:  OK
Image 4 of current batch - Image status:  OK
Image 5 of current batch - Image status:  OK
Image 6 of current batch - Image status:  OK
Image 7 of current batch - Image status:  OK
Image 8 of current batch - Image status:  OK
Image 9 of current batch - Image status:  OK
Image 10 of current batch - Image status:  OK
Image 11 of current batch - Image status:  OK
Image 12 of current batch - Image status:  OK
Image 13 of current batch - Image status:  OK
Image 14 of current batch - Image status:  OK
Image 15 of current batch - Image status:  OK
Image 16 of current batch - Image status:  OK
Image 17 of current batch - Image status:  OK
Image 18 of current batch - Image status:  OK
Image 19 of current batch - Image status:  OK
Image 20 of current batch - Image status:  OK
Image 21 of current batch - Image status:  OK
Image 22 of current batch 

Upload new batch
Image 1 of current batch - Image status:  OK
Image 2 of current batch - Image status:  OK
Image 3 of current batch - Image status:  OK
Image 4 of current batch - Image status:  OK
Image 5 of current batch - Image status:  OK
Image 6 of current batch - Image status:  OK
Image 7 of current batch - Image status:  OK
Image 8 of current batch - Image status:  OK
Image 9 of current batch - Image status:  OK
Image 10 of current batch - Image status:  OK
Image 11 of current batch - Image status:  OK
Image 12 of current batch - Image status:  OK
Image 13 of current batch - Image status:  OK
Image 14 of current batch - Image status:  OK
Image 15 of current batch - Image status:  OK
Image 16 of current batch - Image status:  OK
Image 17 of current batch - Image status:  OK
Image 18 of current batch - Image status:  OK
Image 19 of current batch - Image status:  OK
Image 20 of current batch - Image status:  OK
Image 21 of current batch - Image status:  OK
Image 22 of current batch 

Upload new batch
Image batch upload failed.
Image 1 of current batch - Image status:  OK
Image 2 of current batch - Image status:  OKDuplicate
Image 3 of current batch - Image status:  OK
Image 4 of current batch - Image status:  OK
Image 5 of current batch - Image status:  OK
Image 6 of current batch - Image status:  OK
Image 7 of current batch - Image status:  OK
Image 8 of current batch - Image status:  OK
Image 9 of current batch - Image status:  OKDuplicate
Image 10 of current batch - Image status:  OK
Image 11 of current batch - Image status:  OK
Image 12 of current batch - Image status:  OK
Image 13 of current batch - Image status:  OK
Image 14 of current batch - Image status:  OK
Image 15 of current batch - Image status:  OK
Image 16 of current batch - Image status:  OK
Image 17 of current batch - Image status:  OK
New current tag: n02115641-dingo
Upload new batch
Image 1 of current batch - Image status:  OK
Image 2 of current batch - Image status:  OK
Image 3 of current batch 

Upload new batch
Image 1 of current batch - Image status:  OK
Image 2 of current batch - Image status:  OK
Image 3 of current batch - Image status:  OK
Image 4 of current batch - Image status:  OK
Image 5 of current batch - Image status:  OK
Image 6 of current batch - Image status:  OK
Image 7 of current batch - Image status:  OK
Image 8 of current batch - Image status:  OK
Image 9 of current batch - Image status:  OK
Image 10 of current batch - Image status:  OK
Image 11 of current batch - Image status:  OK
Image 12 of current batch - Image status:  OK
Image 13 of current batch - Image status:  OK
Image 14 of current batch - Image status:  OK
Image 15 of current batch - Image status:  OK
Image 16 of current batch - Image status:  OK
Image 17 of current batch - Image status:  OK
Image 18 of current batch - Image status:  OK
Image 19 of current batch - Image status:  OK
Image 20 of current batch - Image status:  OK
Image 21 of current batch - Image status:  OK
Image 22 of current batch 

# Model Training

In [ ]:
print ("Start training...")
print("=" * 20)
iteration = trainer.train_project(project.id, training_type="Advanced", reserved_budget_in_hours=2)

while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(30)

Start training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training


# Publish Endpoint

The iteration is now trained. Publish it to the project endpoint

In [ ]:
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Endpoint published!")

# Model Evaluation

In [ ]:
base_image_location_test = "../data/test"

correct_pred_list = []
print("Testing model on test set...")

for root, dirs, files in os.walk(base_image_location_test):
    for i, file in enumerate(files): 
        with open(os.path.join(root, file), "rb") as image_contents:
            results = predictor.classify_image(
                project.id, publish_iteration_name, image_contents.read())
            
            # Display the results.
            correct_pred_list.append(results.predictions[0].tag_name == root.split("/")[-1])

test_accuracy = (np.sum(correct_pred_list)/len(correct_pred_list))
print(f"Test Accuracy: {test_accuracy*100}%")